<a href="https://colab.research.google.com/github/ChenshuLiu/Pytorch-Tutorial/blob/main/Pytorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Tutorial
#### Chenshu Liu
Reference: https://youtu.be/c36lUUr864M?list=PLw1A_xmFf3RLPk5cKf1PZo6-3E-boYifg 

In [2]:
import torch

## Tensors Basics
* use `torch.empty(dimension)` to define an empty tensor object

In [3]:
x = torch.empty(2, 3)
print(x)

tensor([[8.6337e-34, 0.0000e+00, 3.5032e-44],
        [0.0000e+00,        nan, 0.0000e+00]])


* use `torch.rand(dimension)` to define a tensor with random numbers with given dimension

In [4]:
x = torch.rand(2, 2)
print(x)

tensor([[0.1364, 0.7621],
        [0.7713, 0.0085]])


* use `torch.ones(dimension, dtype)` to define tensor object with ones

In [5]:
x = torch.ones(2, 2, dtype = torch.double)
print(x)
print(x.dtype)
print(x.size())

tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64)
torch.float64
torch.Size([2, 2])


* use `torch.tensor([array of values])` to cast array of values into a tensor object

In [6]:
x = torch.tensor([2.5, 0.1])
print(x)

tensor([2.5000, 0.1000])


## Tensor Arithematics
* Addition `torch.add(a,b)`
* Subtraction `torch.sub(a,b)`
* Multiplication `torch.mul(a,b)`
* Division `torch.div(a,b)`
* For in-place modification, add an underscore `_` behind the operation name (i.e. `a.add_(b)` will add a to b and assigned the sum to a)
* Without `_` in the operation, the result need to be assigned to a new object (e.g. `c = a.add(b)`)

In [24]:
x = torch.rand(2, 2)
y = torch.rand(2, 2)
# elementwise addition
z = x + y
#  or
z = torch.add(x, y)
print(x)
print(y)
print(z)

tensor([[0.6109, 0.4688],
        [0.5876, 0.3950]])
tensor([[0.5343, 0.4071],
        [0.7797, 0.0901]])
tensor([[1.1452, 0.8760],
        [1.3672, 0.4851]])


In [27]:
z = y.add(x)
print(x)
print(y)
print(z)
# in-place modification on y
y.add_(x)
print(y)

tensor([[0.6109, 0.4688],
        [0.5876, 0.3950]])
tensor([[1.7561, 1.3448],
        [1.9548, 0.8800]])
tensor([[2.3670, 1.8137],
        [2.5424, 1.2750]])
tensor([[2.3670, 1.8137],
        [2.5424, 1.2750]])


## Tensor Slicing

In [14]:
x = torch.rand(3, 3)
print(x[1, 1])
print(type(x[1, 1]))
# extract only value from the position
print(x[1, 1].item())
print(type(x[1, 1].item()))

tensor(0.9280)
<class 'torch.Tensor'>
0.9280080795288086
<class 'float'>


## Tensor Resizing
For unknown dimension, use `-1` and specify the other dimension (works the same in Numpy)

In [23]:
x = torch.rand(4, 4)
print(x.shape)
# -1 here is just a place-holder
y = x.view((-1, 8))
print(y.shape)

torch.Size([4, 4])
torch.Size([2, 8])
